This notebook defines and instantiates a U-Net architecture for image segmentation, which is then trained on the [Oxford pet database](https://www.robots.ox.ac.uk/~vgg/data/pets/).

Code is taken from [here](https://blog.paperspace.com/unet-architecture-image-segmentation/), [here](https://blog.paperspace.com/understanding-canet-architecture/), [here](https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96) and [here](https://keras.io/examples/vision/oxford_pets_image_segmentation/) and tweaked for better explanation.

# U-Net architecture

## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

## Convolution block

In [ ]:
def convolution_operation(entered_input, filters=64):
    # Taking first input and implementing the first conv block
    conv1 = Conv2D(filters, kernel_size = (3,3), padding = "same")(entered_input)
    batch_norm1 = BatchNormalization()(conv1)
    act1 = ReLU()(batch_norm1)

    # Taking first input and implementing the second conv block
    conv2 = Conv2D(filters, kernel_size = (3,3), padding = "same")(act1)
    batch_norm2 = BatchNormalization()(conv2)
    act2 = ReLU()(batch_norm2)

    return act2

## Downsampling block

In [ ]:
def encoder(entered_input, filters=64):
    # Collect the start and end of each sub-block for normal pass and skip connections
    enc1 = convolution_operation(entered_input, filters)
    MaxPool1 = MaxPooling2D(strides = (2,2))(enc1)
    return enc1, MaxPool1

## Upsampling block

In [ ]:
def decoder(entered_input, skip, filters=64):
    # Upsampling and concatenating the essential features
    Upsample = Conv2DTranspose(filters, (2, 2), strides=2, padding="same")(entered_input)
    Connect_Skip = Concatenate()([Upsample, skip])
    out = convolution_operation(Connect_Skip, filters)
    return out

## Whole architecture

In [ ]:
def U_Net(Image_Size, num_classes):
    # Take the image size and shape
    input1 = Input(Image_Size)

    # Construct the encoder blocks
    skip1, encoder_1 = encoder(input1, 64)
    skip2, encoder_2 = encoder(encoder_1, 64*2)
    skip3, encoder_3 = encoder(encoder_2, 64*4)
    skip4, encoder_4 = encoder(encoder_3, 64*8)

    # Preparing the next block
    conv_block = convolution_operation(encoder_4, 64*16)

    # Construct the decoder blocks
    decoder_1 = decoder(conv_block, skip4, 64*8)
    decoder_2 = decoder(decoder_1, skip3, 64*4)
    decoder_3 = decoder(decoder_2, skip2, 64*2)
    decoder_4 = decoder(decoder_3, skip1, 64)

    out = Conv2D(num_classes, 1, padding="same", activation="sigmoid")(decoder_4)

    model = Model(input1, out)
    return model

## Taking a look at the architecture

In [ ]:
input_shape = (160, 160, 3)
model = U_Net(input_shape, num_classes = 20)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, "model.png", show_shapes=False, show_dtype=False, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)

# Pet dataset

## Download


In [ ]:
import os

#data from: https://academictorrents.com/details/b18bbd9ba03d50b0f7f479acc9f4228a408cecc1
#if the links on-the-fly (here: https://www.robots.ox.ac.uk/~vgg/data/pets/) do not work
#we can use the previously downloaded data from jackdellequerce.com
#if one of the file is there we assume everything was already downloaded
if not os.path.isfile('annotations/README'):
  #downloading the archives
  #!curl -O https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
  #!curl -O https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
  !curl -O http://www.jackdellequerce.com/data/oxford-iiit-pet/images.tar.gz
  !curl -O http://www.jackdellequerce.com/data/oxford-iiit-pet/annotations.tar.gz
  #unpacking
  !tar -xf images.tar.gz
  !tar -xf annotations.tar.gz
  #deleting the archives
  !rm images.tar.gz
  !rm annotations.tar.gz

## Data description

In [ ]:
input_dir = "images/"
target_dir = "annotations/trimaps/"
img_size = (160, 160)
num_pet_classes = 3
batch_size = 32

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

#A bit of interface: printing number of images and the first 10 file names
print("Number of samples:", len(input_img_paths))
for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)


## Taking a look at one of the images

In [ ]:
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

# Display input image #7
display(Image(filename=input_img_paths[9]))

# Display auto-contrast version of corresponding target (per-pixel categories)
img = PIL.ImageOps.autocontrast(load_img(target_img_paths[9]))
display(img)


## Sequence helper

In [ ]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img


class OxfordPets(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
            # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
            y[j] -= 1
        return x, y


# Training

## Train/validatin split

In [ ]:
import random

# Split our img paths into a training and a validation set
val_samples = 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = OxfordPets(
    batch_size, img_size, train_input_img_paths, train_target_img_paths
)
val_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)


## Installing segmentation_models module (via pip)

In [ ]:
#This module is required for IoU and Dice scores
!pip install segmentation_models

## Defining the Dice Loss (Jaccard index)

In [ ]:
from keras import backend as K

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))

    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.

    Ref: https://en.wikipedia.org/wiki/Jaccard_index

    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    #added these two casts to force same type
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    #this below is wassname's original code
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return (1 - jac) * smooth

## Model instantiation

In [ ]:
%env SM_FRAMEWORK=tf.keras
import tensorflow as tf
import segmentation_models as sm
from tensorflow.keras import models
from segmentation_models.metrics import iou_score

In [ ]:
num_pet_classes = 3
input_shape = (160, 160, 3)
model = U_Net(input_shape, num_classes = num_pet_classes)

model.compile(optimizer="rmsprop", loss=jaccard_distance_loss, metrics=[iou_score])

## Model training

In [ ]:
# Train the model, doing validation at the end of each epoch.
epochs = 3

history = model.fit(train_gen, epochs=epochs, validation_data=val_gen)

## Plot loss and metrics

In [ ]:
import matplotlib.pyplot as plt

#a handy function to plot loss and metrics
def plot_loss_history(h, title):
  for metric in h.history.keys():
    #ignoring metrics on validation set, which are implied when
    #plotting on training set
    if metric.startswith('val_'):
      continue

    #if we get here we found a metric on the training set,
    #let's plot it
    plt.plot(h.history[metric], label = "Train set")
    plt.plot(h.history["val_" + metric], label = "Validation set")
    plt.xlabel('Epochs')
    plt.title(title + ' - ' + metric)
    plt.legend()
    plt.show()

#let's use the function we just defined
plot_loss_history(history, title = 'U-Net pet segmentation')

## Predict the validation set

In [ ]:
# Generate predictions for all images in the validation set

val_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)
val_preds = model.predict(val_gen)

def display_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    display(img)


# Display results for validation image #10
i = 10

# Display input image
display(Image(filename=val_input_img_paths[i]))

# Display ground-truth target mask
img = PIL.ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)

# Display mask predicted by our model
display_mask(i)  # Note that the model only sees inputs at 150x150.

# Alternative loss function

Keras offers a different loss function which could be used for segmentation: [sparse_categorical_crossentropy()](https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class)

It's:

* categorical: that's expected, since segmentation is intrinsically a categorical problem
* crossentropy: we already met this
* sparse: useful for very unbalanced classes

In [ ]:
model2 = U_Net(input_shape, num_classes = num_pet_classes)
model2.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=[iou_score])
history2 = model2.fit(train_gen, epochs=epochs, validation_data=val_gen)

In [ ]:
#let's use the function we just defined
plot_loss_history(history2, title = 'U-Net pet segmentation')

In [ ]:
val_preds = model2.predict(val_gen)

# Display input image
display(Image(filename=val_input_img_paths[i]))

# Display ground-truth target mask
img = PIL.ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)